In [ ]:
# 필요 라이브러리 설치
%pip install httpx shapely

Note: you may need to restart the kernel to use updated packages.


In [2]:
import httpx
from shapely.geometry import shape, Point
from shapely.strtree import STRtree  # ✅ 핵심: STRtree 추가

# 최신 행정동 데이터 소스
GEOJSON_URL = "https://raw.githubusercontent.com/vuski/admdongkor/master/ver20250101/HangJeongDong_ver20250101.geojson"

class KoreaRegionManager:
    def __init__(self):
        self.features = []      # 전체 데이터(속성 포함) 저장
        self.geometries = []    # 도형(Polygon)만 따로 리스트 (Tree 생성용)
        self.sigungu_set = set()
        self.tree = None        # ✅ R-tree 인덱스 저장 변수

    async def load_data(self):
        """데이터를 다운로드하고 R-tree 인덱스를 생성"""
        print(f"📥 데이터 다운로드 및 파싱 중... ({GEOJSON_URL})")
        try:
            async with httpx.AsyncClient() as client:
                response = await client.get(GEOJSON_URL)
                response.raise_for_status()
                data = response.json()
                
                raw_features = data.get('features', [])
                
                for item in raw_features:
                    props = item['properties']
                    geometry = shape(item['geometry']) 
                    
                    # 시군구 목록 처리
                    full_name = props.get('adm_nm', '')
                    sido = props.get('sidonm', '')
                    sgg = props.get('sggnm', '')
                    if not sgg: sgg = sido

                    self.sigungu_set.add(f"{sido} {sgg}")
                    
                    # 리스트에 저장
                    self.features.append({'properties': props, 'geometry': geometry})
                    self.geometries.append(geometry) # Tree 생성을 위해 도형만 따로 모음
                
                # ✅ R-tree(STRtree) 생성
                # 모든 다각형의 바운딩 박스를 트리 구조로 정리합니다.
                self.tree = STRtree(self.geometries)
                    
                print(f"✅ 데이터 로드 및 인덱싱 완료: 총 {len(self.features)}개의 행정동")
                
        except Exception as e:
            print(f"❌ 데이터 로드 실패: {e}")

    def get_sigungu_list(self):
        return sorted(list(self.sigungu_set))

    def find_region_by_coord(self, lat, lon):
        """R-tree를 사용하여 좌표가 속한 행정구역을 빠르게 검색"""
        if self.tree is None:
            print("⚠️ 데이터가 로드되지 않았습니다.")
            return None

        target_point = Point(lon, lat)
        
        # ✅ 1. Tree 조회 (query)
        # 해당 점이 포함될 '가능성이 있는(바운딩 박스가 겹치는)' 다각형의 인덱스들을 반환합니다.
        # 전체 3500개를 다 도는 게 아니라, 후보 1~3개 정도만 순식간에 찾아냅니다.
        candidate_indices = self.tree.query(target_point)
        
        # ✅ 2. 정밀 검사
        # 후보들 중에서 실제로 점을 포함(contains)하는지 확인
        for index in candidate_indices:
            candidate_geom = self.geometries[index]
            
            if candidate_geom.contains(target_point):
                props = self.features[index]['properties']
                return {
                    "full_name": props.get('adm_nm'),
                    "sido": props.get('sidonm'),
                    "sigungu": props.get('sggnm'),
                    "dong": props.get('adm_nm').split()[-1]
                }
        
        return None

# 매니저 인스턴스 생성
manager = KoreaRegionManager()

In [3]:
# 데이터를 메모리에 로드합니다. (최초 1회 실행 필요)
await manager.load_data()

📥 데이터 다운로드 및 파싱 중... (https://raw.githubusercontent.com/vuski/admdongkor/master/ver20250101/HangJeongDong_ver20250101.geojson)
✅ 데이터 로드 및 인덱싱 완료: 총 3555개의 행정동


In [4]:
# 시군구 리스트 가져오기
sigungu_list = manager.get_sigungu_list()

print(f"📋 총 {len(sigungu_list)}개의 시군구(시/도 + 시/군/구)가 확인되었습니다.\n")

# 앞부분 10개만 예시로 출력
print("--- 리스트 예시 (상위 10개) ---")
for name in sigungu_list[:10]:
    print(name)

📋 총 252개의 시군구(시/도 + 시/군/구)가 확인되었습니다.

--- 리스트 예시 (상위 10개) ---
강원특별자치도 강릉시
강원특별자치도 고성군
강원특별자치도 동해시
강원특별자치도 삼척시
강원특별자치도 속초시
강원특별자치도 양구군
강원특별자치도 양양군
강원특별자치도 영월군
강원특별자치도 원주시
강원특별자치도 인제군


In [7]:
# 테스트할 좌표 입력 (위도, 경도)
# 예: 서울시청
target_lat = 37.234
target_lon = 127.185

# 예: 해운대 해수욕장
# target_lat = 35.1587
# target_lon = 129.1604

print(f"📍 검색 좌표: 위도 {target_lat}, 경도 {target_lon}")

result = manager.find_region_by_coord(target_lat, target_lon)

if result:
    print("\n✅ 검색 결과:")
    print(f"  - 전체 주소: {result['full_name']}")
    print(f"  - 시 / 도  : {result['sido']}")
    print(f"  - 시 / 군 / 구: {result['sigungu']}")
    print(f"  - 읍 / 면 / 동: {result['dong']}")
else:
    print("\n❌ 해당 좌표는 대한민국 행정구역(육지) 내에서 찾을 수 없습니다.")

📍 검색 좌표: 위도 37.234, 경도 127.185

✅ 검색 결과:
  - 전체 주소: 경기도 용인시처인구 역북동
  - 시 / 도  : 경기도
  - 시 / 군 / 구: 용인시처인구
  - 읍 / 면 / 동: 역북동
